In [7]:
import numpy as np
import os
from ase.io import read


In [3]:
A = np.zeros((5,5))
t = 1
# fill with some values everywhere except the diagonal. Keep the diagonal zero
# A +=  np.random.rand(5, 5)
for i in range(5):
    if i+1 < 5:
        A[i, i+1] = t
    A[i, i] = 0.0
A = (A + A.T) / 2.0
A

array([[0. , 0.5, 0. , 0. , 0. ],
       [0.5, 0. , 0.5, 0. , 0. ],
       [0. , 0.5, 0. , 0.5, 0. ],
       [0. , 0. , 0.5, 0. , 0.5],
       [0. , 0. , 0. , 0.5, 0. ]])

In [4]:
eigenvalues, eigenvectors = np.linalg.eigh(A)
eigenvalues

array([-8.66025404e-01, -5.00000000e-01, -5.14855411e-17,  5.00000000e-01,
        8.66025404e-01])

In [5]:
gamma_L = np.zeros((5, 5), dtype=complex)
gamma_R = np.zeros((5, 5), dtype=complex)
gamma_L[0,0] = 0.05
gamma_R[4,4] = 0.05

In [6]:
occupancies = np.ones(5)
np.save("occupancies.npy", occupancies)

In [20]:
def build_ppp_matrix(
    system_root: str,
    num_sites: int,
    *,
    U_onsite: float = 2.0,
    coulomb_evA: float = 14.397,
    symbols: tuple[str, ...] = ("C", "N"),
    xyz_path: str = "./scatt.xyz",
    symmetrize: bool = True,
) -> np.ndarray:
    """
    Build and return a PPP / Ohno-type interaction matrix U (num_sites x num_sites).

    Parameters
    ----------
    system_root : str
        Root directory containing the geometry.
    num_sites : int
        Number of sites (matrix dimension).
    U_onsite : float
        Onsite interaction U (eV).
    coulomb_evA : float
        Coulomb conversion factor e^2/(4πϵ0) in eV·Å.
    symbols : tuple[str, ...]
        Chemical symbols used as sites (default: C, N).
    xyz_path : str
        Relative path to the XYZ geometry file.
    symmetrize : bool
        If True, enforce U = (U + U.T)/2.

    Returns
    -------
    np.ndarray
        PPP interaction matrix U (eV).

    Raises
    ------
    FileNotFoundError
        If the XYZ file is missing.
    ValueError
        If too few atoms are available or parameters are unphysical.
    """
    if num_sites <= 0:
        raise ValueError(f"num_sites must be positive; got {num_sites}")

    if U_onsite <= 0:
        raise ValueError(f"U_onsite must be positive; got {U_onsite}")

    # ---------------------------
    # PPP parameter
    # ---------------------------
    alpha_ppp = (U_onsite / coulomb_evA) ** 2  # 1/Å^2

    # ---------------------------
    # Load geometry
    # ---------------------------
    xyz_full_path = os.path.join(system_root, xyz_path)
    if not os.path.isfile(xyz_full_path):
        raise FileNotFoundError(f"Missing geometry file: {xyz_full_path}")

    atoms = read(xyz_full_path)
    keep = np.isin(atoms.get_chemical_symbols(), list(symbols))
    site_atoms = atoms[keep]

    if len(site_atoms) < num_sites:
        raise ValueError(
            f"Found only {len(site_atoms)} {symbols} atoms, "
            f"but num_sites={num_sites} was requested."
        )

    positions = site_atoms.get_positions()[:num_sites]  # Å

    # ---------------------------
    # Distance matrix r_ij
    # ---------------------------
    disp = positions[:, None, :] - positions[None, :, :]
    r = np.linalg.norm(disp, axis=-1)

    # ---------------------------
    # PPP interaction matrix
    # ---------------------------
    U = U_onsite / np.sqrt(1.0 + alpha_ppp * r**2)
    np.fill_diagonal(U, U_onsite)

    if symmetrize:
        U = 0.5 * (U + U.T)

    return U

In [23]:
num_sites = A.shape[0]

U_ppp = build_ppp_matrix(
    system_root="output",
    num_sites=num_sites,
    U_onsite=4.62,
    coulomb_evA=14.397,
    symbols=("C", "N"),
    xyz_path="scatt.xyz",
    symmetrize=True,
)
np.savetxt("output/U_ppp.txt", U_ppp)